# Exploring and Applying XGBoost

## Abstract:

This project will focus on the famous XGBoost system and its application on moderate and large datasets. Starting with a history of the system, then exploring the algorithm itself, and finally ending with two applications, this project will hopefully provide a framework on which to base further research and applications.

## History:

From Tianqi Chen and Carlos Guestrin in March of 2014, XGBoost has been a monumental system in the field of advanced analytics and machine learning. XGBoost was initially a research project started by Tianqi Chen - and later Carlos Guestrin - at the University of Washington, who presented their paper at SIGKDD Conference in 2016. Since then, version 1.5.0 is available for public usage and has been implemented in more than 5 languages including: C++, Python, R, Java, Scala, and Julia. In addition, the system is available to all modern operating systems, including: Windows, OS X, Linux, and a variety of cloud platforms.

## The XGBoost Algorithm:

Gradient boosting is a machine learning technique for regression and classification problems. The sequential building process of a decision tree usually consists of two loops. The first loop is an outer loop for enumerating the leaf nodes and the second one is an inside of the outer loop that enumerates the features. Instead of sorting the observations of the node by each feature value, you can instead sort the observations first, that is enable global sorting, and then use a scan to decide the best split.

The XGBoost system utilizes a few key system optimizations, including:

1.  **Parallelization:**

    -   The dataset is split into multiple, smaller subsets that are then distributed among multiple CPU cores. Each subset calculates the quantiles in parallel and then each of those data points are pulled in to form a histogram of quantiles.

    -   The weights associated with each quantile is based on the prior probabilities for classification problems.

2.  **Cache-aware Access:**

    -   Essentially, the cached memory within the CPU is the quickest possible way to access data, considering its location in most systems. This is where the $1^{st}$ and $2^{nd}$ derivatives (gradients and hessian matrix) are stored to later calculate the scores for each node and leaf in the tree.

3.  **Regularization:**

    -   The algorithm includes support for both L1 and L2 regularization to prevent over-fitting.

4.  **Exact Greedy Algorithm for Split Finding:**

    -   The XGBoost algorithm utilizes a greedy search approach for finding the optimal splits within the subsets of data, since it is normally impossible to enumerate all of the possible tree structures available.

    -   It should be noted that when dealing with very large datasets, the exact greedy algorithm falls apart since you can't access the cache-aware Hessians and gradients as efficiently.

    -   A greedy algorithm is any procedure that solves a problem by taking the locally-optimal choice at each iteration or stage when asked to present a solution.

------------------------------------------------------------------------

<p style="color:red;font-size:160%">

**Algorithm 1** *Exact Greedy Algorithm for Finding Splits*

</p>

------------------------------------------------------------------------

1.  Let $M_{0}$ denote the *null* model, containing no predictors.

2.  For \$k=0, \dots, p-1\$:

    -   Consider all $p-k$ models that augment model $M_{k}$ with one additional predictor.

    -   Choose the *best* among these $p-k$ models ($\min{(RSS)}$ or $\max{(R^{2})}$.

3.  Select the single best model among the $M_{p}$ possible models using cross validation.

------------------------------------------------------------------------

Boosting fits ensemble models like the following:

$$
\Large f(x) = \sum_{m=0}^{M} f_m(x)
$$

The most common base, weak learners come from other tree algorithms, like decision trees, and the goal of the weak learner is to have high bias and low variance. However, when many of these weak learners are put together, then one gets the added benefit of having a lower bias.

### Important Components for XGBoost to Succeed:

Like many other boosting algorithms, XGBoost relies on 3 keys to success, namely:

1.  A weak, base learner

    -   Simple decision trees are used to fit the training data.

2.  An additive model that reduces the number of failures

    -   By parameterizing each tree that is added to the model, we reduce the residual error and approach the correct direction in the response surface. This methodology is also known as gradient descent.

3.  A loss function

    -   This is typically the MSE for regression problems and is usually the softmax objective function for multi-class classification problems.

## Available Hyper-parameters in Different Implementations:

### Python

### R

### Julia

## Application - Forest Cover Types: {#application---forest-cover-types}

Given forestry data from four wilderness areas in Roosevelt National Forest, classify the patches into one of $7$ cover types, listed below:

1.  Aspen
2.  Cottonwood/Willow
3.  Douglas-fir
4.  Krummholz
5.  Lodgepole Pine
6.  Ponderosa Pine
7.  Spruce/Fir


In [2]:
###############################################################################
###                    1. Import Libraries and Models                       ###
###############################################################################
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold
import statsmodels.api as sm
from sklearn import metrics
from rich import print as rprint

import warnings
warnings.filterwarnings(action="ignore")

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 200)
plt.style.use("ggplot")

Reading in the finalized data set:

In [3]:
df = pd.read_csv("df.csv")
df.columns

Index(['Unnamed: 0', 'Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points', 'WildernessArea',
       'SoilType', 'Cover_Type', 'CoverName'],
      dtype='object')

## Splitting the dataset into training and testing sets:

In [4]:
X = df.drop(columns=["Cover_Type", "CoverName"])
y = df[["Cover_Type"]]

xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2, random_state=1234, shuffle=True)

## Importing the Models to Run Later:

In [5]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

# Add models as you wish
Models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "ADA Boost": AdaBoostClassifier(),
    "SVClassifier": SVC(),
    "KNN": KNeighborsClassifier(),
    "LDA": LinearDiscriminantAnalysis(),
    "XGBoost": GradientBoostingClassifier()
}

## Setting the Hyperparameters:

In [6]:
HyperParams = {}  # Start with an empty dictionary

HyperParams.update({"Multionimial Regression":
                    {"fit_intercept": [True],
                     "max_iter": [100],
                     "verbose": [1],
                     "n_jobs": [-1],
                     "penalty": ["l1"],
                     "C": [1, 10, 50],
                     "solver": ["saga"], # For multi-class classification,
                     "random_state": [1234]
                     }})

HyperParams.update({"Decision Tree":
                    {"criterion": ["gini", "entropy"],
                     "random_state": [1234]
                    }})

HyperParams.update({"Random Forest": {
    "n_estimators": [300, 500],
    "max_features": ["auto", "sqrt"],
    "n_jobs": [-1]
}})

HyperParams.update({"ADA Boost":
                    {"n_estimators": [400],
                     "learning_rate": [0.001, 0.05, 0.10, 0.5]
                     }})

HyperParams.update({"SVClassifier":
                    {"kernel": ["rbf"],
                     "verbose": [True],
                     "gamma": [0.001, 0.0001]
                     }})

HyperParams.update({"KNN":
                    {"n_neighbors": [3],
                     "algorithm": ["auto"],
                     "n_jobs": [-1]
                     }})

HyperParams.update({"LDA":
                    {"solver": ["eigen"],
                     "store_covariance": [False],
                     "shrinkage": [None]
                     }})

HyperParams.update({"XGBoost":
                    {"max_depth": [3, 4, 5, 6, 7, 8],
                    "n_estimators": [500],
                    }})

## Building the Class to run the Models:

The main class for this code comes from David Batista.

I simply modified it a bit to my liking.

[Source](http://www.davidsbatista.net/blog/2018/02/23/model_optimization/)

In [9]:
class EstimatorSelectionHelper:

    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError(
                "Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    @staticmethod
    def check_rows(X, y):  # My addendum
        if X.shape[0] == y.shape[0]:
            rprint("[bold green]Good[/bold green]: X and Y have the same number of rows")
        else:
            rprint("[bold red]Bad[/bold red]: X and Y do not have the same number of rows")

    def fit(self, X, y, cv=3, n_jobs=-2, verbose=1, scoring=None, refit=False):
        times = []
        for key in self.keys:
            print(f"Running GridSearchCV for {key}.")
            start_time = time.time()
            
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X, y)
            self.grid_searches[key] = gs
            
            time_taken = np.round(time.time() - start_time, 4)
            rprint(f"Run complete for [bold blue]{model}[/bold blue]; took [bold green]{np.round(time_taken, 4)}[/bold green] seconds")
            times.append(time_taken)
        return times

    def score_summary(self, sort_by='mean_score'):
        def row(key, scores, params):
            d = {
                'estimator': key,
                'min_score': min(scores),
                'max_score': max(scores),
                'mean_score': np.mean(scores),
                'std_score': np.std(scores),
                "range_score": (max(scores) - min(scores))  # My own metric
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            print(k)
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'min_score', 'mean_score',
                   'max_score', 'std_score', "range_score"]
        columns = columns + [c for c in df.columns if c not in columns]

        return df[columns]

Now, pass in the dictionary of models and hyperparameters to the class.

In [10]:
pipeline = EstimatorSelectionHelper(Models, HyperParams)
pipeline.check_rows(xTrain, yTrain)

Good: X and Y have the same number of rows

## Fitting the models:

In [11]:
myMetrics = ["accuracy"]
pipeline.fit(xTrain, yTrain, cv=2, n_jobs=-2, scoring=myMetrics, verbose=1)

Running GridSearchCV for Logistic Regression.
Fitting 2 folds for each of 3 candidates, totalling 6 fits


Run complete for LogisticRegression(); took 96.1046 seconds

Running GridSearchCV for Decision Tree.
Fitting 2 folds for each of 2 candidates, totalling 4 fits


Run complete for DecisionTreeClassifier(); took 14.9905 seconds

Running GridSearchCV for Random Forest.
Fitting 2 folds for each of 4 candidates, totalling 8 fits


Run complete for RandomForestClassifier(); took 1308.7161 seconds

Running GridSearchCV for ADA Boost.
Fitting 2 folds for each of 4 candidates, totalling 8 fits


Run complete for AdaBoostClassifier(); took 7723.6537 seconds

Running GridSearchCV for SVClassifier.
Fitting 2 folds for each of 2 candidates, totalling 4 fits


## Extracting the Results from the Fitted Models:

In [ ]:
results = pipeline.score_summary(sort_by="mean_score")
results